In [2]:
from ase import Atom, Atoms
from ase.calculators.espresso import Espresso
from ase.constraints import FixAtoms
from ase.io.cif import read_cif
from ase.io.espresso import read_espresso_out, write_espresso_in
from ase.io.vasp import read_vasp
from ase.visualize import view
from ase.visualize.plot import plot_atoms

from copy import deepcopy
from matplotlib import pyplot as plt
import os
import numpy as np
import threading
from datetime import datetime

from qe_utils import (velocity,
    import_vasp,
    output_to_atoms,
    relax,
    pin_bottom_layers,
    get_D_position,
    preview,
    md,
    sanitize)

In [3]:
# Numeric constants
# 1 picosecond = n Rydberg a.u.
PS_TO_AU = 1e-12 / (4.8378 * 1e-17)

# 1 femtosecond
FS_TO_AU = 1e-15 / (4.8378 * 1e-17)

#########################################################################
# RUN_RELAXATION
#    - True: run a relaxation step (can take several hours)
#    - False: don't run a relaxation step, use an existing relaxed crystal
# An existing relaxed crystal exists in relax_data/relax_Hf5Nb2Ta10Zr5.out
#########################################################################
RUN_RELAXATION = False

#########################################################################
# VACUUM
#    - Amount of vacuum, in Angstroms, to place on either side of the system
#########################################################################
VACUUM = 2.0

#########################################################################
# DEUTERIUM_MASS_AMU
#    - Deuterium mass in AMU
#########################################################################
DEUTERIUM_MASS_AMU = 2.014

#########################################################################
# INITIAL_DISTANCE_A
#    - Initial distance between D atom and surface of slab, in Angstroms
#########################################################################
INITIAL_DISTANCE_A = 1.5

#########################################################################
# AXIS
#    - One of { 'x', 'y', 'z' }: axis along which the D atom should travel
#########################################################################
AXIS = 'x'

#########################################################################
# N_STEPS
#    - Number of steps to run MD for
#########################################################################
N_STEPS = 1

#########################################################################
# INITIAL_EV
#    - Initial eV to impart on D atom
#########################################################################
INITIAL_EV = 60

#########################################################################
# DT
#    - dt, in AU
#########################################################################
DT = 0.2 * round(FS_TO_AU) # 0.2fs

In [4]:
# Run relaxation, if needed
if RUN_RELAXATION:
    slab = import_vasp('input/HfNbTaZr_8.vasp', truncate=False)
    slab.center(vacuum=1)
    relax_output_filename = relax(slab)
    
# Create our slab
slab = output_to_atoms(relax_output_filename if RUN_RELAXATION else 'relax_data/relax_Hf5Nb2Ta10Zr5.out') # This slab is the result of relaxing a 22-atom crystal
atoms = deepcopy(slab)
atoms.center(vacuum=VACUUM, axis=2)
atoms = pin_bottom_layers(atoms, nlayers=2, axis=AXIS)

# Place the D atom in the center of the slab, `INITIAL_DISTANCE_A` Angstroms away
DEUTERIUM_XYZ = get_D_position(atoms, INITIAL_DISTANCE_A=INITIAL_DISTANCE_A, axis=AXIS)
deuterium = Atom('H', mass=DEUTERIUM_MASS_AMU, position=DEUTERIUM_XYZ)
atoms.append(deuterium)

# Expand unit cell so that the D atom fits
existing_cell = atoms.get_cell()
atoms.set_cell(np.array([ # TODO this code is dogshit, clean it up
    existing_cell[0][0] + (2 * INITIAL_DISTANCE_A if AXIS == 'x' else 0),
    existing_cell[1][1] + (2 * INITIAL_DISTANCE_A if AXIS == 'y' else 0),
    existing_cell[2][2] + (2 * INITIAL_DISTANCE_A if AXIS == 'z' else 0)]
))

### Try parallelization
Run `N_JOBS` jobs using `N_CORES_PER_JOB` cores each.

In [ ]:
N_JOBS = 6
N_CORES_PER_JOB = 2

def run_job(args):
    i = args
    print(f'Starting run {i} at {datetime.now()}')
    md(atoms, nsteps=N_STEPS, dt=DT, initial_eV=INITIAL_EV, suffix=f'{i}', ncores=N_CORES_PER_JOB)
    print(f'Completed run {i} at {datetime.now()}')

threads = []
for i in range(N_JOBS):
    t = threading.Thread(target=run_job, args=[i])
    threads.append(t)

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

Starting run 0 at 2023-06-07 04:29:26.523945
Starting run 1 at 2023-06-07 04:29:26.526538
Starting run 2 at 2023-06-07 04:29:26.527169
Starting run 3 at 2023-06-07 04:29:26.529386
Starting run 4 at 2023-06-07 04:29:26.530526
Starting run 5 at 2023-06-07 04:29:26.533041
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
Writing D initial velocity 60eV (0.034658164561745024 Hartree au)
